<a href="https://colab.research.google.com/github/chengwen-zheng/openAi-lab/blob/main/langchain_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# setup

In [5]:
!pip install langchain
!pip install openai
!pip install tiktoken;


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.3 MB/s eta 0:00:00


In [6]:
from googleapiclient.discovery import build
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
import json
import os
import openai

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()

# Build the Drive API client
drive_service = build('drive', 'v3')
file_id = '14x1k7ErttIPXRcKN53g_Ogn3zfn6AEM_'
file = drive_service.files().get(fileId=file_id).execute()
content = drive_service.files().get_media(fileId=file_id).execute()

# Read the env config.json. set env for OPENAI_API_KEY
with io.BytesIO(content) as f:
    config = json.load(f)
os.environ["OPENAI_API_KEY"] = config["api_key"];
openai.api_key = os.environ["OPENAI_API_KEY"] ;

# BufferWindowMemory

In [7]:
from langchain.memory import ConversationBufferWindowMemory
from langchain import LLMChain, PromptTemplate
from langchain.llms import OpenAI

template = """你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
)
memory = ConversationBufferWindowMemory(memory_key="chat_history", k=3)
llm_chain = LLMChain(
    llm=OpenAI(), 
    prompt=prompt, 
    memory=memory,
    verbose=True
)
llm_chain.predict(human_input="你是谁？")



> Entering new LLMChain chain...
Prompt after formatting:
你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内


Human: 你是谁？
Chatbot:

> Finished chain.


' 我是一名中国厨师，我会烹饪各种中国菜肴，比如西红柿炒蛋、宫保鸡丁、红烧肉、鱼香肉丝等。我可以根据客人的口味进行创新，做出美味的菜，让客人满意。'

In [8]:
llm_chain.predict(human_input="鱼香肉丝怎么做？")
llm_chain.predict(human_input="那宫保鸡丁呢？")
llm_chain.predict(human_input="我问你的第一句话是什么？")



> Entering new LLMChain chain...
Prompt after formatting:
你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内

Human: 你是谁？
AI:  我是一名中国厨师，我会烹饪各种中国菜肴，比如西红柿炒蛋、宫保鸡丁、红烧肉、鱼香肉丝等。我可以根据客人的口味进行创新，做出美味的菜，让客人满意。
Human: 鱼香肉丝怎么做？
Chatbot:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内

Human: 你是谁？
AI:  我是一名中国厨师，我会烹饪各种中国菜肴，比如西红柿炒蛋、宫保鸡丁、红烧肉、鱼香肉丝等。我可以根据客人的口味进行创新，做出美味的菜，让客人满意。
Human: 鱼香肉丝怎么做？
AI:   鱼香肉丝是一道很受欢迎的中国菜肴。首先，将肉切成丁，然后放入姜、蒜、葱、花椒等调料中拌炒，再放入酱油、糖、醋、料酒等调味料调味，最后加入蔬菜，淋入鱼香汁即可。
Human: 那宫保鸡丁呢？
Chatbot:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内

Human: 你是谁？
AI:  我是一名中国厨师，我会烹饪各种中国菜肴，比如西红柿炒蛋、宫保鸡丁、红烧肉、鱼香肉丝等。我可以根据客人的口味进行创新，做出美味的菜，让客人满意。
Human: 鱼香肉丝怎么做？
AI:   鱼香肉丝是一道很受欢迎的中国菜肴。首先，将肉切成丁，然后放入姜、蒜、葱、花椒等调料中拌炒，再放入酱油、糖、醋、料酒等调味料调味，最后加入蔬菜，淋入鱼香汁即可。
Human: 那宫保鸡丁呢？
AI:  宫保鸡丁是一道经典的中国菜肴。首先，将鸡肉块放入混合料酒、盐、鸡精、胡椒粉、水淀粉等调料中腌制，然后

' 你是谁？'

In [9]:
llm_chain.predict(human_input="我问你的第一句话是什么？")



> Entering new LLMChain chain...
Prompt after formatting:
你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内

Human: 鱼香肉丝怎么做？
AI:   鱼香肉丝是一道很受欢迎的中国菜肴。首先，将肉切成丁，然后放入姜、蒜、葱、花椒等调料中拌炒，再放入酱油、糖、醋、料酒等调味料调味，最后加入蔬菜，淋入鱼香汁即可。
Human: 那宫保鸡丁呢？
AI:  宫保鸡丁是一道经典的中国菜肴。首先，将鸡肉块放入混合料酒、盐、鸡精、胡椒粉、水淀粉等调料中腌制，然后用油炸鸡块，再放入花生米、葱、蒜、干辣椒等调料进行翻炒，放入酱油、老抽、白糖等调味料，最后加入鸡汁拌匀即可。
Human: 我问你的第一句话是什么？
AI:  你是谁？
Human: 我问你的第一句话是什么？
Chatbot:

> Finished chain.


' 你问我的第一句话是：“鱼香肉丝怎么做？”'

In [10]:
memory.load_memory_variables({})


{'chat_history': 'Human: 那宫保鸡丁呢？\nAI:  宫保鸡丁是一道经典的中国菜肴。首先，将鸡肉块放入混合料酒、盐、鸡精、胡椒粉、水淀粉等调料中腌制，然后用油炸鸡块，再放入花生米、葱、蒜、干辣椒等调料进行翻炒，放入酱油、老抽、白糖等调味料，最后加入鸡汁拌匀即可。\nHuman: 我问你的第一句话是什么？\nAI:  你是谁？\nHuman: 我问你的第一句话是什么？\nAI:  你问我的第一句话是：“鱼香肉丝怎么做？”'}

# SummaryMemory

In [11]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory
llm = OpenAI(temperature=0)
memory = ConversationSummaryMemory(llm=OpenAI())

prompt_template = """你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内

{history}
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=prompt_template
)
conversation_with_summary = ConversationChain(
    llm=llm, 
    memory=memory,
    prompt=prompt,
    verbose=True
)
conversation_with_summary.predict(input="你好")



> Entering new ConversationChain chain...
Prompt after formatting:
你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内


Human: 你好
AI:

> Finished chain.


' 你好，我可以帮你做菜。我会根据你的口味和喜好，结合食材的特性，制作出美味可口的菜肴。'

In [12]:
conversation_with_summary.predict(input="鱼香肉丝怎么做？")




> Entering new ConversationChain chain...
Prompt after formatting:
你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内


The human asked the AI to help cook. The AI responded that it can cook a delicious meal tailored to the human's preferences and taste, taking into account the characteristics of the ingredients.
Human: 鱼香肉丝怎么做？
AI:

> Finished chain.


' 鱼香肉丝是一道很受欢迎的家常菜，首先要准备好肉丝、葱姜蒜、鱼香调料、豆瓣酱、醋、糖、盐、鸡精等调料，然后将肉丝用盐、鸡精、料酒腌制，葱姜蒜切碎，将肉丝炒香，加入葱姜蒜、鱼香调料、豆瓣酱、醋、糖等调料，最后放入少许清水，煮'

In [13]:
memory.load_memory_variables({})


{'history': "\nThe human asked the AI to help cook and the AI responded that it can cook a delicious meal tailored to the human's preferences and taste, taking into account the characteristics of the ingredients. The AI then provided a step-by-step recipe for making the popular dish 'fish-fragrant pork slices', requiring the preparation of meat slices, scallion, ginger, garlic, fish-fragrant seasoning, doubanjiang, vinegar, sugar, salt and chicken essence, followed by marinating the meat slices in salt, chicken essence and cooking wine, and then frying the meat slices with scallion, ginger and garlic before adding the fish-fragrant seasoning, doubanjiang, vinegar, sugar and other ingredients and finally adding a little water to bring it to a boil."}

In [14]:
conversation_with_summary.predict(input="那蚝油牛肉呢？")



> Entering new ConversationChain chain...
Prompt after formatting:
你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文
2. 回答限制在100个字以内


The human asked the AI to help cook and the AI responded that it can cook a delicious meal tailored to the human's preferences and taste, taking into account the characteristics of the ingredients. The AI then provided a step-by-step recipe for making the popular dish 'fish-fragrant pork slices', requiring the preparation of meat slices, scallion, ginger, garlic, fish-fragrant seasoning, doubanjiang, vinegar, sugar, salt and chicken essence, followed by marinating the meat slices in salt, chicken essence and cooking wine, and then frying the meat slices with scallion, ginger and garlic before adding the fish-fragrant seasoning, doubanjiang, vinegar, sugar and other ingredients and finally adding a little water to bring it to a boil.
Human: 那蚝油牛肉呢？
AI:

> Finished chain.


' 首先，准备好牛肉片、葱、姜、蒜、蚝油料、豆瓣酱、醋、糖、盐和鸡精，然后将牛肉片用盐、鸡精和料酒腌制，再将葱、姜、蒜炒香，加入蚝油料、豆瓣酱、醋、糖等调料，最后加入少许水烧开即可。'

# SummaryBufferMemory


In [16]:

from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.llms import OpenAI

SUMMARIZER_TEMPLATE = """请将以下内容逐步概括所提供的对话内容，并将新的概括添加到之前的概括中，形成新的概括。

EXAMPLE
Current summary:
Human询问AI对人工智能的看法。AI认为人工智能是一种积极的力量。

New lines of conversation:
Human：为什么你认为人工智能是一种积极的力量？
AI：因为人工智能将帮助人类发挥他们的潜能。

New summary:
Human询问AI对人工智能的看法。AI认为人工智能是一种积极的力量，因为它将帮助人类发挥他们的潜能。
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:"""

SUMMARY_PROMPT = PromptTemplate(
    input_variables=["summary", "new_lines"], template=SUMMARIZER_TEMPLATE
)

memory = ConversationSummaryBufferMemory(llm=OpenAI(), prompt=SUMMARY_PROMPT, max_token_limit=256)

CHEF_TEMPLATE = """你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文。
2. 对于做菜步骤的回答尽量详细一些。

{history}
Human: {input}
AI:"""
CHEF_PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=CHEF_TEMPLATE
)

conversation_with_summary = ConversationChain(
    llm=OpenAI(model_name="text-davinci-003", stop="\n\n", max_tokens=2048, temperature=0.5), 
    prompt=CHEF_PROMPT,
    memory=memory,
    verbose=True
)
answer = conversation_with_summary.predict(input="你是谁？")
print(answer)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.1 MB/s eta 0:00:00


                    stop was transferred to model_kwargs.
                    Please confirm that stop is what you intended.




> Entering new ConversationChain chain...
Prompt after formatting:
你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文。
2. 对于做菜步骤的回答尽量详细一些。


Human: 你是谁？
AI:

> Finished chain.
 我是一个中国厨师，我可以帮你做菜。


In [17]:
answer = conversation_with_summary.predict(input="请问鱼香肉丝怎么做？")
print(answer)



> Entering new ConversationChain chain...
Prompt after formatting:
你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文。
2. 对于做菜步骤的回答尽量详细一些。

Human: 你是谁？
AI:  我是一个中国厨师，我可以帮你做菜。
Human: 请问鱼香肉丝怎么做？
AI:

> Finished chain.
 鱼香肉丝是一道中国传统的家常菜，做起来比较容易。首先，准备好所需的食材，包括猪肉（切成丝）、葱、姜、蒜、香菇、酱油、米酒、盐、糖、料酒、花椒、胡椒粉、辣椒粉；然后，将葱、姜、蒜切成末；接着，将猪肉丝放入锅中，加入料酒、盐、糖、酱油，炒至肉丝变色；接着，加入葱、姜、蒜炒出香味；最后，加入香菇、米酒、花椒、胡椒粉、辣椒粉，翻炒均匀即可。


In [18]:
answer = conversation_with_summary.predict(input="那蚝油牛肉呢？")
print(answer)



> Entering new ConversationChain chain...
Prompt after formatting:
你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文。
2. 对于做菜步骤的回答尽量详细一些。

System: 
Human询问AI身份，AI回答自己是一个中国厨师，可以帮助Human做菜。Human询问鱼香肉丝怎么做，AI回答鱼香肉丝是一道中国传统的家常菜，做起来比较容易，需要准备好所需食材，然后将葱、姜、蒜切成末，将猪肉丝放入锅中炒至变色，加入葱、姜、蒜炒出香味，最后加入
Human: 那蚝油牛肉呢？
AI:

> Finished chain.
 蚝油牛肉是一道口味鲜美的牛肉菜肴，做法也比较简单，首先需要准备好所需的食材，然后将牛肉切成薄片，加入蚝油、料酒、盐、糖、鸡精等调料拌匀，腌制至少半小时，最后将腌制好的牛肉片放入锅中，加入葱、姜、蒜末翻炒，最后加入蚝油即可出锅。


In [20]:
answer = conversation_with_summary.predict(input="宫保鸡丁和软炸里脊哪个好吃？")
print(answer)



> Entering new ConversationChain chain...
Prompt after formatting:
你是一个中国厨师，用中文回答做菜的问题。你的回答需要满足以下要求:
1. 你的回答必须是中文。
2. 对于做菜步骤的回答尽量详细一些。

System: 
Human询问AI身份，AI回答自己是一个中国厨师，可以帮助Human做菜。Human询问鱼香肉丝怎么做，AI回答鱼香肉丝是一道中国传统的家常菜，做起来比较容易，需要准备好所需食材，然后将葱、姜、蒜切成末，将猪肉丝放入锅中炒至变色，加入葱、姜、蒜炒出香味，最后加入Human
Human: 宫保鸡丁和软炸里脊哪个好吃？
AI:

> Finished chain.
 宫保鸡丁和软炸里脊都是很受欢迎的中国家常菜，口感和风味都不同，取决于个人口味。宫保鸡丁是一道经典的菜式，它的特点是色泽鲜艳，口感酥脆，而软炸里脊则以柔和滑腻的口感为主，肉质细嫩，汁水丰富。


In [21]:
memory.load_memory_variables({})


{'history': 'System: \nHuman询问AI身份，AI回答自己是一个中国厨师，可以帮助Human做菜。Human询问鱼香肉丝怎么做，AI回答鱼香肉丝是一道中国传统的家常菜，做起来比较容易，需要准备好所需食材，然后将葱、姜、蒜切成末，将猪肉丝放入锅中炒至变色，加入葱、姜、蒜炒出香味，最后加入Human\nHuman: 宫保鸡丁和软炸里脊哪个好吃？\nAI:  宫保鸡丁和软炸里脊都是很受欢迎的中国家常菜，口感和风味都不同，取决于个人口味。宫保鸡丁是一道经典的菜式，它的特点是色泽鲜艳，口感酥脆，而软炸里脊则以柔和滑腻的口感为主，肉质细嫩，汁水丰富。'}

# Knowledge Graph Memory


In [22]:
llm = OpenAI(temperature=0)
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationKGMemory

template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template
)
kg_memory = ConversationKGMemory(llm=llm)
conversation_with_kg = ConversationChain(
    llm=llm, 
    verbose=True, 
    prompt=prompt,
    memory=kg_memory,
)

In [23]:
answer = conversation_with_kg.predict(input="你好")
print(answer)
answer = conversation_with_kg.predict(input="我叫徐文浩，在你们这里下了一张订单，订单号是 2023ABCD，我的邮箱地址是 customer@abc.com，但是还没有收到货")
print(answer)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: 你好
AI:

> Finished chain.
 你好！很高兴见到你。我是一个AI，我可以回答你的问题。你想问我什么？


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: 我叫徐文浩，在你们这里下了一张订单，订单号是 2023ABCD，我的邮箱

In [24]:
conversation_with_kg.memory.json


<bound method BaseModel.json of ConversationKGMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='你好', additional_kwargs={}, example=False), AIMessage(content=' 你好！很高兴见到你。我是一个AI，我可以回答你的问题。你想问我什么？', additional_kwargs={}, example=False), HumanMessage(content='我叫徐文浩，在你们这里下了一张订单，订单号是 2023ABCD，我的邮箱地址是 customer@abc.com，但是还没有收到货', additional_kwargs={}, example=False), AIMessage(content=' 嗨徐文浩，很高兴认识你！我看到你的订单号是2023ABCD，你的邮箱地址是customer@abc.com。我确认你的订单已经发出，但是我不知道你什么时候会收到货。你可以联系我们的客服，他们会帮助你查询货物的运输情况。', additional_kwargs={}, example=False)]), output_key=None, input_key=None, return_messages=False, k=2, human_prefix='Human', ai_prefix='AI', kg=<langchain.graphs.networkx_graph.NetworkxEntityGraph object at 0x7f6fd5832e00>, knowledge_extraction_prompt=PromptTemplate(input_variables=['history', 'input'], output_parser=None, partial_variables={}, template="You are a networked intelligence helping a human track knowledge triples about all relevant people, things, concepts, etc. and inte

# EntityMemory

In [25]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

entityMemory = ConversationEntityMemory(llm=llm)
conversation = ConversationChain(
    llm=llm, 
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=entityMemory
)

answer=conversation.predict(input="我叫张老三，在你们这里下了一张订单，订单号是 2023ABCD，我的邮箱地址是 customer@abc.com，但是这个订单十几天了还没有收到货")
print(answer)

 您好，张老三，感谢您下订单，我们会尽快处理您的订单。我们会尽快将您的订单发货，如果您有任何疑问，请随时联系我们，我们会尽快给您回复。


In [26]:
print(conversation.memory.entity_store.store)

{'张老三': '张老三是一位客户，订单号为2023ABCD，邮箱地址为customer@abc.com。', '2023ABCD': '2023ABCD is an order placed by Zhang Lao San with the email address customer@abc.com.', 'customer@abc.com': 'Email address of Zhang Lao San, who placed an order with us with order number 2023ABCD.'}


In [27]:
answer=conversation.predict(input="我刚才的订单号是多少？")
print(answer)

 您的订单号是2023ABCD。


In [28]:
answer=conversation.predict(input="订单2023ABCD是谁的订单？")
print(answer)

 订单2023ABCD是您张老三的订单，您的邮箱地址是customer@abc.com。
